## PBT-XL Classification

# Links
DATASET: https://physionet.org/content/ptb-xl/1.0.1/

Project Requirements: https://apmonitor.com/pds/index.php/Main/CourseProject

Github Repository: https://github.com/scraven4/EKG-Classification


In [7]:
!pip install wfdb


In [ ]:
##SAMPLE CODE FROM DATASET INFO PAGE - IMPORTS DATA AND SPLITS INTO TEST/TRAIN SET

import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    #choose 100 for the 100hz file (500mb) and 500hz (2.7gB)
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = r"C:\Users\scrav\Documents\ECE 493R\FInal Project - EKG\PBT-XL Extracted\\"


# load and convert annotation data
Y = pd.read_csv(r"C:\Users\scrav\Documents\ECE 493R\FInal Project - EKG\PBT-XL Extracted\ptbxl_database.csv", index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass